In [1]:
program_text = """

void main(int ars, char argv[])
{
    int i;    //init
    float f;
    string s;
    bool b;
    
    hex h;
    oct o;
    bin bit;
    fixed fix;
    
    read(s);   //read write
    print( s );
    
    i = 06;    //assign
    i =0x7;
    i= 0b01;
    i=6;
    f = 0.50;
    
    if(i == 6  or i==5 and i!= 6 and i<=5)  //condition
    {
        i = 5;
    }
    else /* I want multicomment here
    and noone can stop me 
    do this */
    {
        i = 6;
    }
    
    while(i < 10) //cycle
    {
        i = i + 1;
    }
    
    i = 08;
    i = 0x1g2;
    i = 0b02;
    i = 0abc;
    ias& = 12;
    i = 12abc;
}

"""

dictionary = {
    "void": "function_type",
    "int": "type",
    "float": "type",
    "string": "type",
    "char": "type",
    "bool":  "type",
    "hex": "type",
    "oct": "type",
    "bin": "type",
    "fixed": "type",
    
    "main": "entry_point",
    "(": "bracket_smooth_l",
    ")": "bracket_smooth_r",
    "{": "bracket_curve_l",
    "}": "bracket_curve_r",
    "[": "bracket_rect_l",
    "]": "bracket_rect_r",
    ";": "divider",
    "read": "reader",
    "print": "printer",
    "==": "comparsion",
    "<=": "comparsion",
    ">=": "comparsion",
    "=": "assign",
    "!": "bool not",
    "!=": "comparsion",
    "<": "comparsion",
    ">": "comparsion",
    "+": "math_symbol",
    "-": "math_symbol",
    "*": "math_symbol",
    "//": "comment_start",
    "/": "math_symbol",
    "%": "math_symbol",
    ",": "comma",
    " ": "separator",
    "\n": "new_line",
    
    "if": "condition",
    "else": "condition_else",
    "while": "cycle"
}

In [2]:
LETTERS = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
SYMBOLS = "(){}[];=<>!+-*/%, "
NUMBERS = "0123456789"
STOP_POINTS = " (){}[];=<>!+-*/%,\n"

BIN = "01"
OCT = "01234567"
HEX = "0123456789abcdefABCDEF"

HEX_START = "x"
BIN_START = "b"

POINT = "."

ROW_NAME = "Row "
ROW_COL_DIVIDER_NAME = " / "
COLUMN_NAME = "Col "

WORD_TO_TYPE_DELIMER = " - "
ERROR_NAME = "ERR"
COMMENT_NAME = "Comment"
ID_NAME = "ID"
HEX_NAME = "HEX"
OCT_NAME = "OCT"
BIN_NAME = "BIN"
DEC_NAME = "INT"
FLOAT_NAME = "FLOAT"

In [3]:
class Lexer(object):
    
    def __init__(self, program_text):
        self.text = program_text + "\n"
        self.list = []
    
        
    def run(self, show_states = False, show_spaces = False):
        
        def state_error(symbol, word, row, column):
            if symbol in STOP_POINTS:
                self.list.append([word[0], WORD_TO_TYPE_DELIMER + ERROR_NAME, ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                word[0] = ""
                return state_start(symbol, word, row, column)
            else:
                return state_error
            
        def state_unary_stop_symbol(symbol, word, row, column):
            if dictionary.get(word[0], False):
                if word[0] == " " and not show_spaces:
                    pass
                else:    
                    self.list.append([word[0], WORD_TO_TYPE_DELIMER + dictionary[word[0]], ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
            word[0] = ""
            return state_start(symbol, word, row, column)
        
        def state_undefined_stop_symbol(symbol, word, row, column):
            if symbol == "=":
                return state_dual_stop_symbol
            else:
                if dictionary.get(word[0], False):
                    self.list.append([word[0], WORD_TO_TYPE_DELIMER + dictionary[word[0]], ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                word[0] = ""
                return state_start(symbol, word, row, column)
            
        def state_dual_stop_symbol(symbol, word, row, column):
            if dictionary.get(word[0], False):
                self.list.append([word[0], WORD_TO_TYPE_DELIMER + dictionary[word[0]], ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
            word[0] = ""
            return state_start(symbol, word, row, column)
        
        def state_slash_symbol(symbol, word, row, column):
            if symbol == "/":
                return state_comment
            elif symbol == "*":
                return state_multi_comment
            else:
                if dictionary.get(word[0], False):
                    self.list.append([word[0], WORD_TO_TYPE_DELIMER + dictionary[word[0]], ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                word[0] = ""
                return state_start(symbol, word, row, column)
        
        def state_comment(symbol, word, row, column):
            if symbol == "\n":
                self.list.append([word[0], WORD_TO_TYPE_DELIMER + COMMENT_NAME, ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                word[0] = ""
                return state_start(symbol, word, row, column)
            else:
                return state_comment
            
        def state_multi_comment(symbol, word, row, column):
            if symbol == "*":
                return state_multi_comment_exit
            else:
                return state_multi_comment
            
        def state_multi_comment_exit(symbol, word, row, column):
            if symbol == "/":
                self.list.append([word[0], WORD_TO_TYPE_DELIMER + COMMENT_NAME, ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                word[0] = ""
                return state_start(symbol, word, row, column)
            else:
                return state_multi_comment
            
                
        def state_identifier(symbol, word, row, column):   
            if symbol in LETTERS + NUMBERS + "_":
                return state_identifier
            else:
                if dictionary.get(word[0], False):
                    self.list.append([word[0], WORD_TO_TYPE_DELIMER + dictionary[word[0]], ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                else:
                    self.list.append([word[0], WORD_TO_TYPE_DELIMER + ID_NAME, ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                word[0] = ""
                return state_start(symbol, word, row, column)
            
        def state_hex_number(symbol, word, row, column):
            if symbol in HEX:
                return state_hex_number
            elif symbol in NUMBERS + LETTERS:
                return state_error(symbol, word, row, column)
            else:
                self.list.append([word[0], WORD_TO_TYPE_DELIMER + HEX_NAME, ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                word[0] = ""
                return state_start(symbol, word, row, column)
            
        def state_bin_number(symbol, word, row, column):
            if symbol in BIN:
                return state_bin_number
            elif symbol in NUMBERS + LETTERS:
                return state_error(symbol, word, row, column)
            else:
                self.list.append([word[0], WORD_TO_TYPE_DELIMER + BIN_NAME, ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                word[0] = ""
                return state_start(symbol, word, row, column)
            
        def state_oct_number(symbol, word, row, column):
            if symbol in OCT:
                return state_oct_number
            elif symbol in NUMBERS + LETTERS:
                return state_error(symbol, word, row, column)
            else:
                self.list.append([word[0], WORD_TO_TYPE_DELIMER + OCT_NAME, ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                word[0] = ""
                return state_start(symbol, word, row, column)
            
        def state_float(symbol, word, row, column):
            if symbol in NUMBERS:
                return state_float
            elif symbol in "eE":
                return state_float_exponent
            elif symbol in LETTERS:
                return state_error(symbol, word, row, column) 
            else:
                self.list.append([word[0], WORD_TO_TYPE_DELIMER + FLOAT_NAME, ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                word[0] = ""
                return state_start(symbol, word, row, column)
            
        def state_undefined_float(symbol, word, row, column):
            if symbol in NUMBERS:
                return state_float
            elif symbol in LETTERS:
                return state_error(symbol, word, row, column) 
            else:
                return state_error(symbol, word, row, column)
            
        def state_float_exponent(symbol, word, row, column):
            if symbol in "+-":
                return state_float_exponent_sign
            else:
                return state_error
            
        def state_float_exponent_sign(symbol, word, row, column):
            if symbol in "123456789":
                return state_float_exponent_numbers
            elif symbol in "0":
                return state_float_exponent_zero
            else:
                return state_error
            
        def state_float_exponent_numbers(symbol, word, row, column):
            if symbol in "0123456789":
                return state_float_exponent_numbers
            elif symbol in LETTERS:
                return state_error(symbol, word, row, column) 
            else:
                self.list.append([word[0], WORD_TO_TYPE_DELIMER + FLOAT_NAME, ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                word[0] = ""
                return state_start(symbol, word, row, column)
            
        def state_float_exponent_zero(symbol, word, row, column):
            if symbol in LETTERS + NUMBERS:
                return state_error(symbol, word, row, column) 
            else:
                self.list.append([word[0], WORD_TO_TYPE_DELIMER + FLOAT_NAME, ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                word[0] = ""
                return state_start(symbol, word, row, column)
        
        def state_any_number(symbol, word, row, column):
            if symbol == HEX_START:
                return state_hex_number
            elif symbol == BIN_START:
                return state_bin_number
            elif symbol in OCT:
                return state_oct_number
            elif symbol == POINT:
                return state_float
            elif symbol in NUMBERS + LETTERS:
                return state_error(symbol, word, row, column)
            else:
                self.list.append([word[0], WORD_TO_TYPE_DELIMER + DEC_NAME, ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                word[0] = ""
                return state_start(symbol, word, row, column)
            
        def state_int_or_float(symbol, word, row, column):
            if symbol in NUMBERS:
                return state_int_or_float
            elif symbol == POINT:
                return state_float
            elif symbol in LETTERS:
                return state_error(symbol, word, row, column)
            else:
                self.list.append([word[0], WORD_TO_TYPE_DELIMER + DEC_NAME, ROW_NAME + str(row) + ROW_COL_DIVIDER_NAME + COLUMN_NAME + str(column)])
                word[0] = ""
                return state_start(symbol, word, row, column)
        
        def state_start(symbol, word, row, column):
            if symbol in LETTERS:
                return state_identifier
            elif symbol == "/":
                return state_slash_symbol
            elif symbol in "<>!=":
                return state_undefined_stop_symbol
            elif symbol in "(){}[];+-*%, \n":
                return state_unary_stop_symbol
            elif symbol in "123456789":
                return state_int_or_float
            elif symbol == "0":
                return state_any_number
            elif symbol == ".":
                return state_undefined_float
            else:
                return state_error
        
        state = state_start
        word = [""]
    
        row = 0
        column = -1
        last_symbol = ""
        for i in self.text:
            new_state = state(i, word, row, column)
            
            if show_states:
                print(new_state, i)
            
            word[0] = word[0] + i
            
            state = new_state
            
            if last_symbol == "\n":
                row = row + 1
                column = 0
            else:
                column = column + 1
                
            last_symbol = i
            
            
    def show(self):
        for i in self.list:
            print(i)
            

In [4]:
lex = Lexer(program_text)
lex.run(show_states = True, show_spaces = True)
lex.show()

<function Lexer.run.<locals>.state_unary_stop_symbol at 0x000002586522FB80> 

<function Lexer.run.<locals>.state_unary_stop_symbol at 0x000002586522FB80> 

<function Lexer.run.<locals>.state_identifier at 0x00000258652550D0> v
<function Lexer.run.<locals>.state_identifier at 0x00000258652550D0> o
<function Lexer.run.<locals>.state_identifier at 0x00000258652550D0> i
<function Lexer.run.<locals>.state_identifier at 0x00000258652550D0> d
<function Lexer.run.<locals>.state_unary_stop_symbol at 0x000002586522FB80>  
<function Lexer.run.<locals>.state_identifier at 0x00000258652550D0> m
<function Lexer.run.<locals>.state_identifier at 0x00000258652550D0> a
<function Lexer.run.<locals>.state_identifier at 0x00000258652550D0> i
<function Lexer.run.<locals>.state_identifier at 0x00000258652550D0> n
<function Lexer.run.<locals>.state_unary_stop_symbol at 0x000002586522FB80> (
<function Lexer.run.<locals>.state_identifier at 0x00000258652550D0> i
<function Lexer.run.<locals>.state_identifier at 

In [5]:
lex = Lexer("iden!ifier")
lex.run()
lex.show()

['iden', ' - ID', 'Row 0 / Col 3']
['!', ' - bool not', 'Row 0 / Col 4']
['ifier', ' - ID', 'Row 0 / Col 9']


In [6]:
lex = Lexer(".278392\n2.432\n8741.")
lex.run()
lex.show()

['.278392', ' - FLOAT', 'Row 0 / Col 6']
['\n', ' - new_line', 'Row 0 / Col 7']
['2.432', ' - FLOAT', 'Row 1 / Col 4']
['\n', ' - new_line', 'Row 1 / Col 5']
['8741.', ' - FLOAT', 'Row 2 / Col 4']


In [7]:
lex = Lexer("3.14e+74\n 3.14e+0\n 3.14e-1222\n 3.14e-\n 3.14e+12.\n 3.14e+12a\n .\n")
lex.run()
lex.show()

['3.14e+74', ' - FLOAT', 'Row 0 / Col 7']
['\n', ' - new_line', 'Row 0 / Col 8']
['3.14e+0', ' - FLOAT', 'Row 1 / Col 7']
['\n', ' - new_line', 'Row 1 / Col 8']
['3.14e-1222', ' - FLOAT', 'Row 2 / Col 10']
['\n', ' - new_line', 'Row 2 / Col 11']
['3.14e-\n', ' - ERR', 'Row 3 / Col 7']
['3.14e+12', ' - FLOAT', 'Row 4 / Col 8']
['.', ' - ERR', 'Row 4 / Col 9']
['\n', ' - new_line', 'Row 4 / Col 10']
['3.14e+12a', ' - ERR', 'Row 5 / Col 9']
['\n', ' - new_line', 'Row 5 / Col 10']
['.', ' - ERR', 'Row 6 / Col 1']
['\n', ' - new_line', 'Row 6 / Col 2']
